In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('6_clear.csv')

In [7]:
df

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,HBondAcceptorCount,RotatableBondCount,IsotopeAtomCount,AtomStereoCount,DefinedAtomStereoCount,UndefinedAtomStereoCount,BondStereoCount,DefinedBondStereoCount,UndefinedBondStereoCount,CovalentUnitCount
0,0.000000,0.000000,0.000000,0.000000,28.478428,2.065207,4.130413,28.478428,1.196505,3.991631,...,0.000000,2.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,2.000000,0.000000
1,11.899854,10.409025,2.000000,0.000000,19.191646,2.196073,4.392146,19.191646,1.199478,3.652157,...,5.000000,5.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
2,1.414214,1.414214,0.000000,0.000000,2.828427,1.414214,2.828427,2.828427,0.942809,1.849457,...,0.000000,3.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,31.900000
3,23.748222,18.952656,1.000000,3.000000,38.424285,2.447029,4.894057,38.424285,1.239493,4.348367,...,6.000000,9.000000,0.000000,2.000000,2.00000,0.000000,0.000000,0.000000,0.000000,1.000000
4,1.414214,1.414214,0.000000,0.000000,2.828427,1.414214,2.828427,2.828427,0.942809,1.849457,...,0.000000,1.800000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.800000,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,12.647042,10.934887,0.000000,1.000000,22.445285,2.334414,4.668828,22.445285,1.320311,3.736834,...,5.000000,17.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000
1067,34.387982,27.357746,0.000000,0.000000,57.122957,2.487084,4.974167,57.122957,1.269399,4.724201,...,21.000000,10.000000,0.000000,20.000000,0.00000,20.000000,0.000000,0.000000,0.000000,1.000000
1068,2.449490,2.449490,0.000000,0.000000,3.464102,1.732051,3.464102,3.464102,0.866025,2.178059,...,3.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
1069,11.247571,9.342944,0.488423,0.443219,19.174214,1.961103,3.895626,19.174214,1.088673,3.056468,...,3.505568,6.562361,0.181514,1.103563,0.85078,0.198218,0.066815,0.075724,0.604677,2.772973
